# CPSC 4300/6300-001 Applied Data Science (Fall 2020)

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

# CPSC4300/6300-001 Problem Set #3

# Part C. Prepare Data for Machine Learning

## 0. Set up environment 

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

## 1. Get the data

In [ ]:
housing = pd.read_csv("input/housing.csv")
attribute_caps = housing[['median_income', 'housing_median_age', 'median_house_value']].max()
housing.head()

In [ ]:
attribute_caps

In [ ]:
housing.info()

## 2. Create Train-Test Split Using Stratified Shuffle Split

Normally, you can use the __train_test_split()__ method in the __sklearn.model_selection__ model to split the a data set data. 

However, a simple random split approach sometime may be not ideal. As shown in the histograms below, most of the samples are concentrated in the regions where median_income is in the range of [1, 8]. When we draw training data overwhelmingly from a concentrated region, the trained machine learning model may not work well with other regions.

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(housing, test_size=0.33, random_state=6300)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
for idx, col in enumerate(['median_income', 'median_house_value']):
    for df in [housing, train_data, test_data]:
        _ = sns.distplot(df[col], bins=20, ax=axes[idx])
    _ = axes[idx].set_xlim(housing[col].min(), housing[col].max())

Ideally, you may want each stratum of the median income has an adequate representation in the training and test datasets. The stratified sampling method serves this purpose. 

In scikit_learn, the __sklearn.model_selection.StratifiedShuffleSplit__ create stratified splits (see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html).

__Complete the following code to create a stratified split of training and testing data according to the median_income and compute the means and percents of each bin as shown in the above code.__

Hint: 
+ You can set n_splits=1 in the __StratifiedShuffleSplit__.
+ __StratifiedShuffleSplit.split(X, y)__ generate indices to split data into training and test set. The stratification is based on the __y__ label. For this question, what would you pass as the __y__ argument?

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Cut data by median_income
bins = [0., 1.5, 3.0, 4.5, 6.0, 8.0, 10.0, np.inf]
housing['income_level'] = pd.cut(housing['median_income'], bins, labels=[1, 2, 3, 4, 5, 6, 7])


# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
df_income_levels = pd.DataFrame()
# Get median_income mean and count at each income level
for df in [housing, train_data, test_data]:
    df1 = df.groupby('income_level').agg({'median_income': ['mean', 'count']})
    df_income_levels = pd.concat([df_income_levels, df1], axis=1)

# Print median_income mean and percentage of samples at each income level
df_income_levels.columns = pd.MultiIndex.from_product([['all_data', "train_data", "test_data"], ["mean", "count"]])
income_level_counts = df_income_levels[('all_data', 'count')]
for dataset in ['all_data', 'train_data', 'test_data']:
    df_income_levels[(dataset, 'count')] = df_income_levels[(dataset, 'count')]/income_level_counts
df_income_levels.columns = pd.MultiIndex.from_product([['all_data', "train_data", "test_data"], ["mean", "percent"]])
df_income_levels

# 3. Handle Missing Values

You may have noticed that the `total_bedrooms` contains some missing data. There are several options to handle attributes with missing values:

1. Remove the rows that contain missing features.
2. Remove the columns that contain missing features.
3. Replace the missing values with new values such as the mean, the median, etc.

The DataFrame methods `dropna()`, `drop()`, and `fillna()` can be used to handle missing data in a DataFrame.

Assume you choose the replacement option, you may consider a systematic solution that handle the following two scenarios:

+ Both the train data and test data need to handle the missing values.
+ When your system is going online, attributes that currently do not contain missing values may have missing values.

The `SimpleImputer` class in the scikit-leran library (https://scikit-learn.org/stable/modules/impute.html) can accomplish this goal. 

__Complete the following code to replace all missing values in each column with the means of that column in the train_data and save the cleaned data into a DataFrame named `df_num`.__

Hint: Because `SimpleImputer` is applicable to numerical attributes only, the DataFrame you will appply a `SimpleImputer` transform must not contain any non-numerical attribute. You can create a new DataFrame that contains all the numerical columns.

In [ ]:
from sklearn.impute import SimpleImputer

# get all numeric columns
df_num = train_data[train_data.dtypes[train_data.dtypes == 'float64'].index]

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test if there are still missing values
all(df_num.isna().sum() == 0)

In [ ]:
# Test if the number of samples has been changed
train_data.shape[0] == df_num.shape[0]

In [ ]:
# Test if the means of the train_data and df_num are same
assert np.abs(train_data['total_bedrooms'].mean() - df_num['total_bedrooms'].mean()) <= 1e-4

## 4. Add Combined Attributes

When you build a machine learning model, you may often find that a derived attribute from combing two raw attributes can have a stronger relationship with the target variable. 

In the housing data, there are four aggregated counts attributes: `total_rooms`, `total_bedrooms`, `population`, and `household`. Intuitively, `median_house_value` will have a stronger relation with  a ratio like `rooms_per_household` than with an aggregation like `total_room`. 

You can write a program  to test which combination may have a strong correlation as follows.

In [ ]:
attrs = ['total_rooms', 'total_bedrooms', 'population', 'households', 'median_house_value']
df_test = df_num[attrs].copy()
derived_attrs = []
for attr1 in attrs[:-1]:
    for attr2 in attrs[:-1]:
        if attr2 == attr1:
            continue
        derived_attr = attr1+'_'+attr2
        derived_attrs.append(derived_attr)
        df_test[derived_attr] = df_test[attr1]/df_test[attr2]

In [ ]:
candidate_attrs = attrs + derived_attrs
corrs = df_test.corr()['median_house_value'].drop(index='median_house_value').sort_values(ascending=False)
print(corrs)

In [ ]:
_ = corrs.plot.barh(title='Correlation Coefficient b/t attribute with median_house_value')

Answer the following questions based on the above results.

__Question 4(a)__. Among `households`, `population`, `household_size`, and `household_population_ratio`, which one has the strongest relation with `'median_house_value`? (Note: `household_size = population/households`) Assign your answer to a string variable `answer`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
answer 

__Question 4(b)__. Does a higher `room_bedroom_ratio` increase the `median_house_value` or decrease the `median_house_value`? Assign your answer (either `increase` or `descrease`) to a string variable `answer`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
answer 

__Question 4(c)__. Assume you decide to add the three derive attributes, `rooms_per_bedroom`,  `household_per_population`, and `rooms_per_household` to `df_num`, write some code to achieve that goal.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
df_num.head(2)

In [ ]:
assert all([col in df_num for col in ['rooms_per_bedroom', 'household_per_population', 'rooms_per_household']])

In [ ]:
assert np.abs(df_num['household_per_population'].mean() - (df_num['households']/df_num['population']).mean()) < 1.0e-6

## 5. Handling Text and Categorical Attributes

Categorical data is one type of structured data we frequently see in data science. For example, the position of a football player, the weather of a day, the size of a shirt, the genres of music and country name are all categorical data.

In categorical data, the values of a data attribute are discrete and belong to a finite set of groups, which are often known as classes or labels. The values of a categorical data can be numeric or textual.

There are two major classes of categorical data: nominal and ordinal. 

+ For nominal categorical attribute, there is no concept of ordering among the values of the attribute. For example, The types of weathers are nominal data.

+ For ordinal categorical attribute, you can place the attributes into some kind of order or scale. For example, you can rate the happiness on a scale of 1-10. Similarly, shoe sizes, education level, income levels, and hurricane categories are examples of ordinal data.

__Question 5(a)__. Is __ocean_proximity__ `nominal categorical data` or `ordinal categorical data`? Assign your answer to a string variable `answer`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
answer

In [ ]:
assert any(['ordinal' in answer.lower(), 'nominal' in answer.lower()])
# There is a hidden test here
assert any(['nominal' in answer.lower()])

__Question 5(b)__. There are multiple ways to encode categorical data such as using `pandas.get_dummies()` method or `sklearn.preprocessing.OneHotEncoder` (see https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

__Complete the code that convert the __ocean_proximity__ categorical data into one-hot vectors using `sklearn.preprocessing.OneHotEncoder`. Save the encoded results to a variable named __housing_cat_1hot__.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = train_data[["ocean_proximity"]]
# YOUR CODE HERE
raise NotImplementedError()
print(housing_cat_1hot[:4])

In [ ]:
import scipy
assert isinstance(housing_cat_1hot, scipy.sparse.csr.csr_matrix)

In [ ]:
assert housing_cat_1hot.shape[1] == len(train_data["ocean_proximity"].value_counts())

## 6. Remove Capped Data

During exploratory data analysis, we have found that the maximum values of several attributes are capped. It might be a good idea to drop those samples.

In [ ]:
_ = train_data[['median_income', 'housing_median_age', 'median_house_value']].hist(bins=50, layout=(1, 3), figsize=(18, 5))

__Question 6(a)__. __Write some code to drop the samples whose attribute values are capped.__

Hint: You can use the `DataFrame['attr'].max()` method to find the upper cap of the attribute 'attr'.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert all([not any(train_data[col] >= attribute_caps[col])
            for col in ['median_house_value', 'housing_median_age', 'median_income']])

Now replot the histogram to validate that those samples are removed.

In [ ]:
_ = train_data[['median_income', 'housing_median_age', 'median_house_value']].hist(bins=50, layout=(1, 3), figsize=(18, 5))

## 7. Feature Scaling

With few exceptions, machine learning algorithms don’t perform well when the input numerical attributes have different scales. For the housing data, the total number of rooms ranges from 6 to 39,320, while the median income ranges from 0 to 15.

There are two common approaches to scale the attributes: min-max scaling and standardization.

Min-max scaling is quite simple: values are shifted and rescaled so that they end up ranging from 0 to 1.

Standardization is different: first, it subtracts the mean value (so standardized values always have a zero mean); then it divides the difference by the standard deviation so that the resulting distribution has unit variance.

__Question 7(a)__. __Write some code to standardize the housing train data using `sklearn.preprocessing.StandardScaler`__. Save the transformed data into a variable `num_tr`__.

In [ ]:
from sklearn.preprocessing import StandardScaler
df_num = train_data.drop("ocean_proximity", axis=1)
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
df = pd.DataFrame(num_tr)
df.columns = df_num.columns
df.head(2)

In [ ]:
assert isinstance(num_tr, np.ndarray)

In [ ]:
# test mean is 0
assert all(np.abs(df.mean()) < 1e-6)

In [ ]:
# test stand deviation is 1
assert all(df.std() - 1 < 1e-3)

## 8. Put things together

Preparing a dataset for machine learning is a tedious process, which involves multiple steps in a specific order. Creating a pipeline or write a data clean module is one way to automate this process.

Initially, I planned to ask you to implement a full pipeline using the Pipeline class in the `scikit-learn` library but found a few issues in the implementation. Therefore, I provide a reference solution to implement a basic data cleaning task and then invoke them in a sequence. Although you are not required to write the code in this assignment, you may still go through the code so that you know how to start when you implement a data cleaning process in your own project. 

In [ ]:
# 0. import required modules
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedShuffleSplit

# 1. Get the data
housing = pd.read_csv("input/housing.csv")
assert isinstance(housing, pd.DataFrame)

In [ ]:
# 2. Separate numeric and categorical features
num_attribs = [col for col in housing.columns if col not in ["median_house_value", "ocean_proximity"]]
cat_attribs = ["ocean_proximity"]
ocean_proximity_categories =  list(housing["ocean_proximity"].value_counts().index)
attribute_caps = housing[['median_income', 'housing_median_age', 'median_house_value']].max()

In [ ]:
# 3. Remove capped values
def remove_capped_samples(df):
    # Record attribute cap
    indices = np.logical_and((df['median_house_value'] < attribute_caps['median_house_value']).values,
                             (df['housing_median_age'] < attribute_caps['housing_median_age']).values,
                             (df['median_income'] < attribute_caps['median_income']).values)
    df = df[indices]
    return df.reset_index(drop=True)

In [ ]:
housing = remove_capped_samples(housing)
housing.head()

In [ ]:
# 4. Impute missing values
def impute_missing_values(df, numeric_attribs):
    imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    _ = imputer.fit(df[numeric_attribs])
    df_num = pd.DataFrame(imputer.transform(df[numeric_attribs]), columns=numeric_attribs)
    return df_num

In [ ]:
df_num = impute_missing_values(housing, num_attribs)
df_num

In [ ]:
# 5. Adds combined attributes
def add_combined_attributes(df):
    df['rooms_per_household'] = df['total_rooms'] / df['households']
    df['population_per_household'] = df['population'] / df['households']
    df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']
    return df

In [ ]:
df_num = add_combined_attributes(df_num)
df_num.head()

In [ ]:
# 6. Encode categorical data
def encode_categorical_data(df, cat_attribs):
    df_cat = None
    for attr in cat_attribs:
        if not df_cat:
            df_cat = pd.get_dummies(df[attr], prefix=attr)
        else:
            df_cat = pd.concat(df_cat, pd.get_dummies(df[attr], prefix=attr))
    return df_cat

In [ ]:
df_cat = encode_categorical_data(housing, cat_attribs)
df_cat.head()

In [ ]:
# 7. Combine column transfermations
def transform_columns(df):
    target = df['median_house_value']
    
    df_num = impute_missing_values(df, num_attribs)
    add_combined_attributes(df_num)
    
    df_cat = encode_categorical_data(df, cat_attribs)
      
    df = pd.concat([df_num, df_cat, target], axis=1)
    
    return df

housing = transform_columns(housing)
housing.head()
    

In [ ]:
# 8. Split data into training data and testing data
def stratified_split(df, test_size=0.25, random_state=0):
    # Cut data by median_income
    bins = [0., 1.5, 3.0, 4.5, 6.0, 8.0, 10.0, np.inf]
    df['income_level'] = pd.cut(df['median_income'], bins, labels=range(1, len(bins)))

    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(df, df["income_level"]):
        train_data_ = df.iloc[train_index].copy()
        test_data_ = df.iloc[test_index].copy()
        break
    for dataset_ in (train_data_, test_data_):
        dataset_.drop("income_level", axis=1, inplace=True)

    # Reset the index to simplify later columns joins
    train_data_ = train_data_.reset_index(drop=True)
    test_data_ = test_data_.reset_index(drop=True)
    return train_data_, test_data_

In [ ]:
housing_train, housing_test = stratified_split(housing, 0.25, 6300)
housing_train.head()

In [ ]:
housing_test.head()

In [ ]:
# 9. Save the cleaned data
housing_train.to_csv("input/housing_train_cleaned.csv", index=False)
housing_test.to_csv("input/housing_test_cleaned.csv", index=False)

In [ ]:
# 10. Create scaled data
def scale_cleaned_data():
    df_train = pd.read_csv("input/housing_train_cleaned.csv")  

    columns_to_scale = df_train.columns[0:11]
    columns_no_sacle = df_train.columns[11:]
    
    scaler = StandardScaler()
    scaler.fit(df_train[columns_to_scale])
    
    num_tr = scaler.transform(df_train[columns_to_scale])
    df_num_tr = pd.DataFrame(num_tr, columns=columns_to_scale)
    df_train = pd.concat([df_num_tr, df_train[columns_no_sacle]], axis=1)
    df_train.to_csv("input/housing_train_scaled.csv", index=False)
    
    df_test = pd.read_csv("input/housing_test_cleaned.csv")
    num_tr = scaler.transform(df_test[columns_to_scale])
    df_num_tr = pd.DataFrame(num_tr, columns=columns_to_scale)
    df_test = pd.concat([df_num_tr, df_test[columns_no_sacle]], axis=1)
    df_test.to_csv("input/housing_test_scaled.csv", index=False)
    

scale_cleaned_data()

In [ ]:
import os
assert all([os.path.exists("input/housing_{}_cleaned.csv".format(fn)) for fn in ['train', 'test']])
assert all([os.path.exists("input/housing_{}_scaled.csv".format(fn)) for fn in ['train', 'test']])

In [ ]:
df_train_1 = pd.read_csv('input/housing_train_cleaned.csv')
df_test_1 = pd.read_csv('input/housing_test_cleaned.csv')
assert df_test_1.shape[1] == df_train_1.shape[1]

In [ ]:
# Test if there are missing data
assert all(df_test_1.isna())

In [ ]:
# Test if a combined attributes exists
assert 'rooms_per_household' in df_test_1.columns

In [ ]:
# Test if categorical features are transformed correctly
assert all([sorted(list(df_test_1.iloc[::, pos].unique())) == list([0, 1]) for pos in range(11,16)])

### Short Questions

__Question 8(a)__. In the features scaling step, we didn't scale the traget variable. __Could you explain why?__ (3 points)

# Type your answer here

__Question 8(b)__. In the above example, we use the feature means to replace the missing values in the data. This works for the numeric features. For categorical features, what would you do for the missing data? (3 points)

# Type your answer here

__Question 8(c)__. In the feature scaling step, I use the cleaned train data to fit a standard scaler and then apply the scaler to scale both train data and test data. Could you explain why I should not fit a new standard scaler from the test data and use such scaler to scale the test data? 

# Type your answer here

__Question 8(d)__. Another subtle issue occurs when applying the OneHotEncode() to train data and test data separately but not all the categorical levels are included in both data sets. Can you explain why this is a problem?

# Type your answer here

__End of Part C__